In [1]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googletrans import Translator
from dotenv import load_dotenv
from contextlib import closing
from tqdm import tqdm
import pandas as pd
import numpy as np
import requests
import sqlite3
import time
import os

In [2]:
_=load_dotenv()

In [3]:
api_key = os.environ['api_key']


In [4]:
#establishing connection with YouTube
youtube = build('youtube', 'v3', developerKey = api_key)

In [5]:
#list of channels to extract
channel_list = ['blockchaindailynews', 'brainbrocrypto', 'cryptoMOC','grenadetw', 'desmondcrypto']

In [6]:
#creating a function to get the channel ids 

def get_channel_ids(youtube, channel_list):
    channel_dict = {}  # Dictionary to store channel names and IDs

    for channel in channel_list:
        # Search for the channel using the channel name
        search_response = youtube.search().list(
            q=channel,
            part='id',
            type='channel',
            maxResults=1
        ).execute()

        # Extract the channel ID from the search results
        channel_id = search_response['items'][0]['id']['channelId']
        channel_dict[channel] = channel_id
        time.sleep(1)

    return channel_dict


In [7]:
channel_dict = get_channel_ids(youtube, channel_list)
channel_dict

{'blockchaindailynews': 'UCkPc41kPIJfMI0ZCTHqybYA',
 'brainbrocrypto': 'UC1EG67JfgUKaZ4Z_l4euycw',
 'cryptoMOC': 'UCv1A3TfyyeOg5q43J9pnpOw',
 'grenadetw': 'UCKyoG3_GoNdCgg5sDoDnnNg',
 'desmondcrypto': 'UCmqS0zPC0zg4L4TiYyTiK4A'}

In [8]:
unique_channel_ids =list(channel_dict.values())
unique_channel_ids                  

['UCkPc41kPIJfMI0ZCTHqybYA',
 'UC1EG67JfgUKaZ4Z_l4euycw',
 'UCv1A3TfyyeOg5q43J9pnpOw',
 'UCKyoG3_GoNdCgg5sDoDnnNg',
 'UCmqS0zPC0zg4L4TiYyTiK4A']

In [9]:

# Establishing/connecting to the SQLite database
connection = sqlite3.connect('Youtube_Analysis.db')

# Creating a cursor object to execute SQL commands
cursor = connection.cursor()

# Creating 'channel_stats' table
cursor.execute('''CREATE TABLE IF NOT EXISTS channel_stats (
                    channel_id TEXT PRIMARY KEY,
                    channel_name TEXT,
                    channel_playlist TEXT,
                    subscriber_count INTEGER,
                    video_count INTEGER
                )''')

# Save the changes and close the connection
connection.commit()
connection.close()

In [11]:
#creating a function to extract channel statistics
def get_channel_stats(youtube, unique_channel_ids):
    all_channel_data = []
    for i in range(len(unique_channel_ids)):
        request = youtube.channels().list(
            part = 'snippet, contentDetails, statistics',
            id = unique_channel_ids)
        response = request.execute()

        for i in range(len(response['items'])):
            channel_data = dict(Channel_id = response['items'][i]['id'],
                            Channel_name = response['items'][i]['snippet']['title'], 
                            Channel_playlist =  response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
                            Subscriber_count = response['items'][i]['statistics']['subscriberCount'],
                            Video_count = response['items'][i]['statistics']['videoCount'])
            all_channel_data.append(channel_data)
            
        #establishig connection
        connection = sqlite3.connect('Youtube_Analysis.db')
        cursor = connection.cursor()

        # Check if the channel ID already exists in the table
        for i in range(len(all_channel_data)):
            cursor.execute("SELECT channel_id FROM channel_stats WHERE channel_id=?", (all_channel_data[i]['Channel_id']),)
            existing_channel_id = cursor.fetchone(i+1)
        
            if existing_channel_id:
                # Update the existing row with the new values
                cursor.execute('''UPDATE channel_stats SET
                                    channel_name = ?,
                                    channel_playlist = ?,
                                    subscriber_count = ?,
                                    video_count = ?
                                  WHERE channel_id = ?''',
                               (channel_name, channel_playlist, subscriber_count, video_count, channel_id))
            else:
                # Insert a new row with the extracted channel details
                cursor.execute('''INSERT INTO channel_stats
                                    (channel_id, channel_name, channel_playlist, subscriber_count, video_count)
                                  VALUES (?, ?, ?, ?, ?)''',
                               (channel_id, channel_name, channel_playlist, subscriber_count, video_count))
        
            # Save the changes and close the connection
            connection.commit()
            connection.close()
            


In [10]:
def update_channel_stats(youtube, unique_channel_ids):
    all_channel_data = []

    # Establish the connection and create the cursor
    connection = sqlite3.connect('Youtube_Analysis.db')
    cursor = connection.cursor()

    for i in range(len(unique_channel_ids)):
        request = youtube.channels().list(
            part='snippet, contentDetails, statistics',
            id=unique_channel_ids[i])
        response = request.execute()

        for i in range(len(response['items'])):
            channel_data = dict(Channel_id=response['items'][i]['id'],
                                Channel_name=response['items'][i]['snippet']['title'],
                                Channel_playlist=response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
                                Subscriber_count=response['items'][i]['statistics']['subscriberCount'],
                                Video_count=response['items'][i]['statistics']['videoCount'])
            all_channel_data.append(channel_data)

            # Check if the channel ID already exists in the table
            cursor.execute("SELECT channel_id FROM channel_stats WHERE channel_id=?", (channel_data['Channel_id'],))
            existing_channel_id = cursor.fetchone()

            if existing_channel_id:
                # Update the existing row with the new values
                cursor.execute('''UPDATE channel_stats SET
                                    channel_name = ?,
                                    channel_playlist = ?,
                                    subscriber_count = ?,
                                    video_count = ?
                                  WHERE channel_id = ?''',
                               (channel_data['Channel_name'], channel_data['Channel_playlist'],
                                channel_data['Subscriber_count'], channel_data['Video_count'], channel_data['Channel_id']))
            else:
                # Insert a new row with the extracted channel details
                cursor.execute('''INSERT INTO channel_stats
                                    (channel_id, channel_name, channel_playlist, subscriber_count, video_count)
                                  VALUES (?, ?, ?, ?, ?)''',
                               (channel_data['Channel_id'], channel_data['Channel_name'],
                                channel_data['Channel_playlist'], channel_data['Subscriber_count'],
                                channel_data['Video_count']))

    # Save the changes and close the connection
    connection.commit()
    connection.close()
    return all_channel_data

In [12]:
channel_database= update_channel_stats(youtube, unique_channel_ids)
channel_database

[{'Channel_id': 'UCkPc41kPIJfMI0ZCTHqybYA',
  'Channel_name': '區塊鏈日報',
  'Channel_playlist': 'UUkPc41kPIJfMI0ZCTHqybYA',
  'Subscriber_count': '44000',
  'Video_count': '502'},
 {'Channel_id': 'UC1EG67JfgUKaZ4Z_l4euycw',
  'Channel_name': '腦哥 Chill塊鏈',
  'Channel_playlist': 'UU1EG67JfgUKaZ4Z_l4euycw',
  'Subscriber_count': '141000',
  'Video_count': '257'},
 {'Channel_id': 'UCv1A3TfyyeOg5q43J9pnpOw',
  'Channel_name': '墨山貓MØC',
  'Channel_playlist': 'UUv1A3TfyyeOg5q43J9pnpOw',
  'Subscriber_count': '38700',
  'Video_count': '120'},
 {'Channel_id': 'UCKyoG3_GoNdCgg5sDoDnnNg',
  'Channel_name': 'GRENADE 手榴彈',
  'Channel_playlist': 'UUKyoG3_GoNdCgg5sDoDnnNg',
  'Subscriber_count': '14900',
  'Video_count': '93'},
 {'Channel_id': 'UCmqS0zPC0zg4L4TiYyTiK4A',
  'Channel_name': 'Desmond的Web3日記',
  'Channel_playlist': 'UUmqS0zPC0zg4L4TiYyTiK4A',
  'Subscriber_count': '41000',
  'Video_count': '193'}]

In [15]:
channel_database_df = pd.DataFrame(channel_database)

In [16]:
#getting playlist containing the entire video_ids of all channels
channel_playlists = list(channel_database_df['Channel_playlist'])
print(channel_playlists)


['UUkPc41kPIJfMI0ZCTHqybYA', 'UU1EG67JfgUKaZ4Z_l4euycw', 'UUv1A3TfyyeOg5q43J9pnpOw', 'UUKyoG3_GoNdCgg5sDoDnnNg', 'UUmqS0zPC0zg4L4TiYyTiK4A']


In [18]:

#creating a function to extract all the videos from the channels
def get_all_videos(playlists):
    all_video_ids= []
    for playlist_id in playlists:
        playlist_videos = []
        next_page_token = None

        while True:
            request = youtube.playlistItems().list(
                part = 'contentDetails',
                playlistId = playlist_id,
                maxResults = 50,
                pageToken=next_page_token
            )                
            response = request.execute()

            for i in range(len(response['items'])):
                video_id = response['items'][i]['contentDetails']['videoId']
                playlist_videos.append(video_id)
                    
            next_page_token = response.get('nextPageToken')               
            if not next_page_token:
                 break  

        all_video_ids.append(playlist_videos)
    return all_video_ids
                
        

In [19]:
video_list= list(get_all_videos(channel_playlists))
video_list


[['yds7hFo47Ck',
  'E4nQuP0jDSI',
  '_SkvmQXQm5o',
  'CjBdKBn0bag',
  'QxFjKtP4Tp4',
  'tE6O2QsmRgM',
  '44fBZ92XC5c',
  'kUs0KPix5s0',
  'D-wS7DGT3p0',
  'nfK1yAAqnqs',
  'EvZ3PoAQyCw',
  '4eavwXKshD4',
  'cth7rVymOjk',
  '_oyyLpigYMQ',
  '7JWDSspgrD4',
  '5VjD8d_2ZLo',
  '-epfEbd2MUw',
  's0SFvW04WbM',
  'VmTxwROlILU',
  'RlURRTduCfQ',
  'dW5CfYmLEQw',
  'jNZY3JRLwS4',
  'UyZSfzVwZes',
  'e_1Ug6TGvik',
  'nG1L9GQcNaI',
  'Id056PsA_Ug',
  'YVgljEQ1cbc',
  'XE-WufhJnog',
  'AziGQOUi3BY',
  'GABoh7C96OE',
  'UUDlPbLmjco',
  'xzXPXVZGMFw',
  'YbCFk-hTWvc',
  '5EU30KijzDc',
  'z8jdGI7m888',
  'yaRp2gcfpNc',
  '0oszU3WyT9Q',
  'lXXDoAvPC2s',
  'KiAQRCag8C0',
  'eS8FhMlUbxQ',
  'KritXl1Bp3c',
  'zn5DZ_YoPKA',
  'WeZtMnjG35o',
  'fQVTXToBo5o',
  'DMYnPVtyXjU',
  'M0tQrAg8dmI',
  'Texy0wVweAE',
  'xnUNvKapBIM',
  'm9tbvxpy8ZU',
  'AiQu1Zx4_lg',
  '5RHq-5OY5tc',
  'tyxGLFinTqU',
  'gWa6C0uNMGw',
  'tuNx0Rb37UQ',
  'lrfjSIC_3Mw',
  'K1pvxaWRhlI',
  '2ZxWm4amoPk',
  'Qmco2WX6j7k',
  'RNQi_3HIufE

In [20]:
all_video_list = list(np.concatenate(video_list))
len(all_video_list)

1167

In [ ]:
#creating a function to extract video data from video ids
def get_video_details(youtube,videos):
    video_details = []
    for items in videos:
        page_token = None

        while True:
            request = youtube.videos().list(
                part = 'snippet, contentDetails, statistics',
                id = items,
                maxResults = 50,
                pageToken = page_token
            )
            response = request.execute()

            video_details.extend(response['items'])
            page_token = response.get('nextPageToken')

            if not page_token:
                break

        all_details = {'video_ids': [],
            'descriptions': [],
            'titles': [],
            'view_counts': [],
            'channel_ids': [],
            'publish_dates': [],
            'tags': [],
            'thumbnails': []
        }
    
        for video in video_details:
            video_id = video['id']
            snippet = video['snippet']
            statistics = video['statistics']

            description = snippet.get('description', '')
            title = snippet.get('title', '')
            view_count = statistics.get('viewCount', '')
        #like_count = statistics.get('likeCount', '')
        #dislike_count = statistics.get('dislikeCount', '')
            channel_id = snippet.get('channelId', '')
            publish_date = snippet.get('publishedAt', '')
            tag = snippet.get('tags','')
            thumbnail = snippet['thumbnails'].get('default', '')

            all_details['video_ids'].append(video_id)
            all_details['descriptions'].append(description)
            all_details['titles'].append(title)
            all_details['view_counts'].append(view_count)
            all_details['channel_ids'].append(channel_id)
            all_details['publish_dates'].append(publish_date)
            all_details['tags'].append(tag)
            all_details['thumbnails'].append(thumbnail)

    return all_details
    

In [ ]:

# Establishing/connecting to the SQLite database
connection = sqlite3.connect('Youtube_Analysis.db')

# Creating a cursor object to execute SQL commands
cursor = connection.cursor()

# Creating 'channel_stats' table
cursor.execute('''CREATE TABLE IF NOT EXISTS video_stats (
                    video_id TEXT PRIMARY KEY,
                    channel_name TEXT,
                    channel_playlist TEXT,
                    subscriber_count INTEGER,
                    video_count INTEGER
                )''')

# Save the changes and close the connection
connection.commit()
connection.close()